In [ ]:
import math
import scipy as sp
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt

from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel, RBF
from random import Random
from numpy import *
from time import *
from datetime import *
from IPython.display import display
from time import time

%matplotlib inline
%load_ext autoreload
%autoreload 2

testing_number = 100

for testing_matrix in range(6, 6+7):

    call = 1
    put = 1

    
    x_train = np.array(np.linspace(0.01,1.0, testing_matrix**3), dtype='float32').reshape(testing_matrix**3, 1)
    x_test = np.array(np.linspace(0.01,1.0, testing_number), dtype='float32').reshape(testing_number, 1)

    y_train = []

    for idx in range(len(x_train)):
        y_train.append(1)
    y_train = np.array(y_train)

    sk_kernel = RBF(length_scale=1.0, length_scale_bounds=(0.01, 10000.0)) #+ WhiteKernel(noise_level = 1e-12) #100000.0
    gp = gaussian_process.GaussianProcessRegressor(kernel=sk_kernel, n_restarts_optimizer=20)
    gp.fit(x_train,y_train)
    y_pred, sigma_hat = gp.predict(x_test, return_std=True)

    l = gp.kernel_.length_scale
    rbf= gaussian_process.kernels.RBF(length_scale=l)

    Kernel= rbf(x_train, x_train)
    K_y = Kernel + np.eye(testing_matrix**3) * 1e-8
    #L, lower = sp.linalg.cho_factor(K_y, lower=True)
    
    t1 = time()
    L1 = sp.linalg.cholesky(K_y, lower=True)
    t2 = time()
    runtime = t2 - t1
    
    print(" ")
    print("Matrix size is {size}.".format(size=L1.shape))
    print("Relative error is {error}".format(error=np.linalg.norm(L1@L1.T-K_y)/np.linalg.norm(K_y)))
    print("Runtime of factorization is {rt} seconds.".format(rt=runtime))
    print(" ")
